In [11]:
import numpy as np 
import pandas as pd 
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime
import kaleido

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

fig = px.choropleth(datafile, 
                    locations='usa_state_code', 
                    color="total_votes",
                    color_continuous_scale="PuOr", 
                    range_color=(0, 10000000),
                    locationmode = 'USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Counts' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.write_image("votes.png")

fig.show()


![Figure](map_png/votes.png)

In [12]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
pop = pd.read_csv('/Users/kellyg/maps-project/data/states_pop.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

votes_percapita = datafile.merge(pop, on='usa_state_code')

#votes per capita
votes_percapita['votes_per_capita'] = votes_percapita['total_votes'] / votes_percapita['population']

fig = px.choropleth(votes_percapita, 
                    locations='usa_state_code', 
                    color="votes_per_capita", 
                    color_continuous_scale="PuOr", 
                    locationmode='USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Per Capita' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.write_image("vote_capita.png")

fig.show()


![Figure](map_png/vote_capita.png)

# Note:

Maps are interactive (zoom and hover for numerical data) when jupyter files ran, but GitHub does not support this feature, so png versions of maps were added. 

In [22]:

import warnings
warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM','REP'])]
df_president_county = df_president_county.groupby(['state','party'])["total_votes"].sum()
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)
df_president_county_dummy = pd.get_dummies(df_president_county['party'])

df_president_county = df_president_county.join(df_president_county_dummy)
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes'] 
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes'] 
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum()
df_president_county = df_president_county.reset_index()
df_president_county['percent_democrat'] = df_president_county['DEM_votes']*100/(df_president_county['REP_votes']+df_president_county['DEM_votes'])

fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',
                    scope="usa",
                    title='2020 Votes By Party')

# fig.write_image("vote_party.png")

fig.show()

![Figure](map_png/vote_party.png)